In [5]:
from sportsipy.ncaab.teams import Teams
from sportsipy.ncaab.schedule import Schedule
from sportsipy.ncaab.boxscore import Boxscore
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup, Comment

In [6]:
BoxscoreIndex= '2021-12-04-20-gonzaga' #2022-02-08-19-arkansas
def find_team_2022(BoxscoreIndex):

    # f string for urls 
    #getting url and soupifying it
    url = f'https://www.sports-reference.com/cbb/boxscores/{BoxscoreIndex}.html'
    f = requests.get(url)
    soup = BeautifulSoup(f.text)


    #scrapping away team records 
    #eg. (15-6)
    away_team_record = str(soup.find_all('li')[35]).split('(')[1].replace(')</a></li>','')
    home_team_record = str(soup.find_all('li')[37]).split('(')[1].replace(')</a></li>','')
    date = str(soup.find_all('h2')[1]).split('—')[1].replace('</h2>','')

    #having issues pulling location

    #location = str(soup.find_all('div')[0].find_all('div')[15].find_all('div')[15].find_all('div')[14].find_all('div')[1]).replace('<div>','').replace('</div>','')

    #scrapping school team names and abrevations
    #Want to change scraping from this to scrapping from score table below 
    school_name_loser = str(soup.find_all('tr', {'class':"loser"})[3].find('a')).split('.')[0].replace('<a href="/cbb/schools/','').replace('/2022','')
    school_name_winner = str(soup.find_all('tr', {'class':"winner"})[3].find('a')).split('.')[0].replace('<a href="/cbb/schools/','').replace('/2022','')
    school_abr_winner = str(soup.find_all('tr', {'class':"winner"})[3].find('a')).split('>')[1].replace('</a','')
    school_abr_loser = str(soup.find_all('tr', {'class':"loser"})[3].find('a')).split('>')[1].replace('</a','')

    score = pd.read_html(str(soup.find_all(text=lambda text: text and isinstance(text, Comment) and 'Line Score Table' in text)))[0]

    away_school = list(score)[0][0][1]
    home_school = list(score)[0][0][0]

    #removing - because they were scrapped from a url
    for letter in school_name_loser:
        if letter == '-':
            letter.replace('-',' ')
        else:
            continue


    for letter in school_name_winner:
        if letter == '-':
            letter.replace('-',' ')
        else:
            continue


    #not all teams have records so using try and except for weird cases
    try:
        away_ranking = str(soup.find_all('span', {'class':'pollrank'})[0]).split('>')[1].replace('\xa0','').replace('(','').replace(')</span','')
    except:
        away_ranking = np.nan
    try:
        home_ranking = str(soup.find_all('span', {'class':'pollrank'})[1]).split('>')[1].replace('\xa0','').replace('(','').replace(')</span','')
    except:
        home_ranking = np.nan

    #records
    home_wins = home_team_record.split('-')[0]
    away_wins = away_team_record.split('-')[0]
    away_losses = away_team_record.split('-')[1]
    home_losses = home_team_record.split('-')[1]
    away_win_percentage = round(int(away_wins) / (int(away_wins)+int(away_losses)),2)
    home_win_percentage = round(int(home_wins) / (int(home_wins)+int(away_losses)),2)


    #advanced_stats
    advanced_stats=pd.read_html(url, match= 'Advanced Box Score Stats',header=0, skiprows =1)


    away_advanced_stats_totals = advanced_stats[0].set_index('Starters', inplace = True)
    away_advanced_stats = advanced_stats[0].loc[['School Totals']].reset_index()


    home_advanced_stats_totals = advanced_stats[1].set_index('Starters', inplace = True)
    home_advanced_stats = advanced_stats[1].loc[['School Totals']].reset_index()


    #basic stats
    basic_stat = pd.read_html(url, match= 'Basic Box Score Stats', index_col=[0])
    away_basic_stat_totals = basic_stat[0].droplevel(level=0, axis=1)
    home_basic_stat_totals = basic_stat[1].droplevel(level=0, axis=1)
    home_basic_stat = home_basic_stat_totals.loc[['School Totals']].reset_index()
    away_basic_stat = away_basic_stat_totals.loc[['School Totals']].reset_index()



    df = pd.DataFrame([school_abr_loser, school_abr_winner, date, #location, 
                       school_name_winner, school_name_loser, away_ranking, home_ranking, 
                       home_wins, away_wins, away_losses, home_losses, away_win_percentage, home_win_percentage]).T

    df.columns = ['losing_abbr','winning_abbr','date', #'location,'
                  'losing_name', 'winning_name', 'away_ranking', 'home_ranking',
                  'home_wins','away_wins', 'home_losses', 'away_losses', 'away_win_percentage', 'home_win_percentage']

    df = pd.concat([df, away_advanced_stats, home_advanced_stats, home_basic_stat, away_basic_stat], axis=1)
    return(df)

In [7]:
find_team_2022(BoxscoreIndex)

,losing_abbr,winning_abbr,date,losing_name,winning_name,away_ranking,home_ranking,home_wins,away_wins,home_losses,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,UALR,ARK,"Dec 4, 2021",arkansas,arkansas-little-rock,18,21,7,7,1,...,.727,9,30,39,15,7,9,12,24,91


In [ ]:

#NEEDS TO ENSURE THAT NAN VALUE IF NOT RANKED

away_ranking = str(soup.find_all('span', {'class':'pollrank'})[0]).split('>')[1].replace('\xa0','').replace('(','').replace(')</span','')
home_ranking = str(soup.find_all('span', {'class':'pollrank'})[0]).split('>')[1].replace('\xa0','').replace('(','').replace(')</span','')
home_ranking

In [ ]:
school_abr_loser = str(loser.find('a')).split('>')[1].replace('</a','')
school_abr_winner = str(winner.find('a')).split('>')[1].replace('</a','')

In [ ]:
school_name_winner = str(winner.find('a')).split('.')[0].replace('<a href="/cbb/schools/','').replace('/2022','')
school_name_loser = str(loser.find('a')).split('.')[0].replace('<a href="/cbb/schools/','').replace('/2022','')

In [ ]:
date = str(soup.find_all('h2')[1]).split('—')[1].replace('</h2>','')
location = str(soup.find_all('div')[0].find_all('div')[15].find_all('div')[15].find_all('div')[14].find_all('div')[1])

In [ ]:
away_team_record = str(soup.find_all('div')[0].find_all('div')[15].find_all('div')[5].find_all('div')[0].find_all('div')[4]).replace('<div>','').replace('</div>','')
home_team_record = str(soup.find_all('div')[0].find_all('div')[15].find_all('div')[5].find_all('div')[7].find_all('div')[4]).replace('<div>','').replace('</div>','')

In [ ]:
home_wins = home_team_record.split('-')[0]
away_wins = away_team_record.split('-')[0]
away_losses = away_team_record.split('-')[1]
home_losses = home_team_record.split('-')[1]
away_win_percentage = round(int(away_wins) / (int(away_wins)+int(away_losses)),2)
home_win_percentage = round(int(home_wins) / (int(home_wins)+int(away_losses)),2)

In [ ]:
html = 'https://www.sports-reference.com/cbb/boxscores/2022-02-04-19-seton-hall.html'
advanced_stats = pd.read_html(url, match= 'Advanced Box Score Stats')
basic_stat = pd.read_html(url, match= 'Basic Box Score Stats')
score = pd.read_html(url)[0]

In [ ]:
advanced_away.drop(columns= ['MP', 'USG%', 'BPM'])

advanced_away.rename(columns={'TS%': 'away_true_shooting_percentage',
                         'eFG%': 'away_effective_field_goal_percentage',
                         '3PAr': 'away_three_point_attempt_rate',
                         'FTr': 'away_free_throw_attempt_rate',
                         'ORB%': 'away_offensive_rebound_percentage',
                         'DRB%': 'away_defensive_rebound_percentage',
                         'TRB%': 'away_total_rebound_percentage',
                         'AST%': 'away_assist_percentage',
                         'STL%': 'away_steal_percentage', 
                         'BLK%': 'away_block_percentage',
                         'TOV%': 'away_turnover_percentage',
                         'ORtg': 'away_offensive_rating',
                         'DRtg': 'away_defensive_rating'        
                })

In [ ]:
advanced_home.drop(columns= ['MP', 'USG%', 'BPM'])

advanced_home.rename(columns={'TS%': 'home_true_shooting_percentage',
                         'eFG%': 'home_effective_field_goal_percentage',
                         '3PAr': 'home_three_point_attempt_rate',
                         'FTr': 'home_free_throw_attempt_rate',
                         'ORB%': 'home_offensive_rebound_percentage',
                         'DRB%': 'home_defensive_rebound_percentage',
                         'AST%': 'home_assist_percentage',
                         'STL%': 'home_steal_percentage', 
                         'BLK%': 'home_block_percentage',
                         'TOV%': 'home_turnover_percentage',
                         'ORtg': 'home_offensive_rating',
                         'DRtg': 'home_defensive_rating'        
                })

In [ ]:
basic_away.rename(columns={'MP': 'away_minutes_played',
                 'FG': 'away_field_goals',
                 'FGA': 'away_field_goal_attempts',
                 'FG%': 'away_field_goal_percentage',
                 '2P': 'away_two_point_field_goals',
                 '2PA': 'away_two_point_field_goal_attempts',
                 '2P%': 'away_two_point_field_goal_percentage',
                 '3P': 'away_three_point_field_goals',
                 '3PA': 'away_three_point_field_goal_attempts',
                 '3P%': 'away_three_point_field_goal_percentage',
                 'FT': 'away_free_throws',
                 'FTA': 'away_free_throw_attempts',
                 'FT%': 'away_free_throw_percentage',
                 'ORB': 'away_offensive_rebounds',
                 'DRB': 'away_defensive_rebounds',
                 'TRB': 'away_total_rebounds',
                 'AST': 'away_assists',
                 'STL': 'away_steals',
                 'BLK': 'away_blocks',
                 'TOV': 'away_turnovers',
                 'PF': 'away_personal_fouls',
                 'PTS': 'away_points',
                 })

In [ ]:
basic_home.rename(columns={'MP': 'home_minutes_played',
                 'FG': 'home_field_goals',
                 'FGA': 'home_field_goal_attempts',
                 'FG%': 'home_field_goal_percentage',
                 '2P': 'home_two_point_field_goals',
                 '2PA': 'home_two_point_field_goal_attempts',
                 '2P%': 'home_two_point_field_goal_percentage',
                 '3P': 'home_three_point_field_goals',
                 '3PA': 'home_three_point_field_goal_attempts',
                 '3P%': 'home_three_point_field_goal_percentage',
                 'FT': 'home_free_throws',
                 'FTA': 'home_free_throw_attempts',
                 'FT%': 'home_free_throw_percentage',
                 'ORB': 'home_offensive_rebounds',
                 'DRB': 'home_defensive_rebounds',
                 'TRB': 'home_total_rebounds',
                 'AST': 'home_assists',
                 'STL': 'home_steals',
                 'BLK': 'home_blocks',
                 'TOV': 'home_turnovers',
                 'PF': 'home_personal_fouls',
                 'PTS': 'home_points',
                 })